In [ ]:
%pylab inline

In [ ]:
import ee
ee.Initialize()

import rendvi
from rendvi import eeCollections
import datetime

In [ ]:
mod = eeCollections.MOD09GQ['imageCollection']
mod1km = eeCollections.MOD09GA['imageCollection']

In [ ]:
masked = rendvi.Masking.applyModis(mod,mod1km)
withNdvi = rendvi.Utils.addNDBand(masked,
                                   b1=eeCollections.MOD09GQ['nir'],
                                   b2=eeCollections.MOD09GQ['red'],
                                   outName='ndvi')

In [ ]:
# time information to handle image collection
iniYear = 2000
endYear = 2019

# convert start and end dates to EE date objects
eeIni = ee.Date.fromYMD(iniYear,1,1,)
eeEnd = ee.Date.fromYMD(endYear,12,31)

# make list of years for loop processing
years = ee.List.sequence(iniYear,endYear)

In [ ]:
full = rendvi.Rendvi(withNdvi,'ndvi')

In [ ]:
dekads = full.getDekadImages(years,includeQa=False)

In [ ]:
climo = dekads.calcClimatology()

In [ ]:
landMask = ee.Image("users/kelmarkert/public/landMask").select("land")
climo = climo.map(lambda img: img.updateMask(landMask))

In [ ]:
exportAsset = "projects/servir-e-sa/reNDVI_climatology"
exportRegion = ee.Geometry.Rectangle([-180,-60,180,85],'epsg:4326',False)
metadataDict = dict(contact="kel.markert@nasa.gov",scale=0.0001,offset=0,version=0)
pyramidingDict = dict(ndvi_mean="mean",ndvi_std="mean",count="mode")

In [ ]:
rendvi.batchExport(climo.limit(30,"system:time_start"), 
                   exportRegion, 
                   exportAsset, 
                   prefix="MOD_NDVI_CLIMATOLOGY", 
                   suffix="v0", 
                   scale=250, 
                   crs='EPSG:4326',
                   metadata=metadataDict, 
                   pyramiding=pyramidingDict
                  )